In [ ]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount=True)

In [1]:
!pip install gdown

# create directory for storing data
!mkdir -p images

# download zip file with training set
!gdown https://drive.google.com/uc?id=1rTf4DJI2PoM-hgSSSzvpOQcQhui3qh-y && unzip -qq train.zip -d images
!rm train.zip

# download zip with valid set
!gdown https://drive.google.com/uc?id=1yB-KN--FjiWZrA6XwxaSvtXhtcbk80dQ && unzip -qq valid.zip -d images
!rm valid.zip

# download zip with utils
!gdown https://drive.google.com/uc?id=1LbwczJVpDLsMGS7yGMP7z1GCAMg4NCsS && unzip -qq utils.zip
!rm utils.zip

# change dir to the one with data 
!cd /content

/bin/sh: gdown: command not found


In [1]:
'''Colorization autoencoder
The autoencoder is trained with grayscale images as input
and colored images as output.
Colorization autoencoder can be treated like the opposite
of denoising autoencoder. Instead of removing noise, colorization
adds noise (color) to the grayscale image.
Grayscale Images --> Colorization --> Color Images
'''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import os


from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, InputLayer, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave, imread, imshow
from skimage.transform import resize

import random

In [2]:
IMG_SIZE = 224
N_EPOCHS = 10
BATCH_SIZE = 32
latent_dim = 256

## Preparing the Data

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True)

train_set = train_datagen.flow_from_directory('images/train_small/',
                                              target_size = (IMG_SIZE, IMG_SIZE),
                                              batch_size = BATCH_SIZE)

valid_datagen = ImageDataGenerator(rescale = 1./255)

valid_set = valid_datagen.flow_from_directory('images/valid_small/',
                                              target_size = (IMG_SIZE, IMG_SIZE),
                                              batch_size = BATCH_SIZE)

def gen_ab_images(train_set):
    for batch in train_set:
        lab_batch = rgb2lab(batch[0])
        X_batch = lab_batch[:,:,:,0] / 100
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)



Found 100 images belonging to 1 classes.
Found 100 images belonging to 1 classes.


## Model

In [3]:
model = Sequential()
model.add(InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Flatten())
model.add(Dense(latent_dim))
model.add(Dense(14*14*256))
model.add(Reshape((14, 14, 256)))
model.add(Conv2DTranspose(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2DTranspose(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2DTranspose(2, (3, 3), activation='relu', padding='same', strides=2))       
model.compile(optimizer='rmsprop', loss='mse')

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 111, 111, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 256)       295168    
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               12845312  
_________________________________________________________________
dense_1 (Dense)              (None, 50176)             1

## Training the model

In [17]:
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit(x=gen_ab_images(train_set), 
          callbacks=[tensorboard], 
          epochs=N_EPOCHS, 
          validation_data=gen_ab_images(valid_set), 
          steps_per_epoch=len(train_set),
          validation_steps=len(valid_set))

Train for 4 steps, validate for 4 steps
Epoch 1/100
4/4 [==============================] - 9s 2s/step - loss: 0.0203 - val_loss: 0.0154
Epoch 2/100
4/4 [==============================] - 9s 2s/step - loss: 0.0197 - val_loss: 0.0166
Epoch 3/100
4/4 [==============================] - 8s 2s/step - loss: 0.0192 - val_loss: 0.0151
Epoch 4/100
4/4 [==============================] - 6s 2s/step - loss: 0.0199


KeyboardInterrupt: 

## Evaluation

In [14]:
batch = next(gen_ab_images(valid_set))
Xtest, Ytest = batch[0], batch[1]

Found 100 images belonging to 1 classes.


In [16]:
print(model.evaluate(Xtest, Ytest, batch_size=BATCH_SIZE))

10/10 [==============================] - 1s 85ms/step
0.01884119212627411


In [30]:
# Test model
output = model.predict(Xtest)
output = output * 128

# Output colorizations

from random import randrange
i = randrange(len(output))
cur = np.zeros((IMG_SIZE, IMG_SIZE, 3))
cur[:,:,0] = Xtest[i][:,:,0] * 100
cur[:,:,1:] = output[i]
cur = lab2rgb(cur)
imshow(cur)

/Users/eryklewinson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: result/img_0.jpg is a low contrast image
  # Remove the CWD from sys.path while we load stuff.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/eryklewinson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: result/img_1.jpg is a low contrast image
  # Remove the CWD from sys.path while we load stuff.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/eryklewinson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: result/img_2.jpg is a low contrast image
  # Remove the CWD from sys.path while we load stuff.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/eryklewinson/anaconda3/lib/python3.7/site-packages/ipykernel